In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

from train_evaluate import NMR_prediction
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop
from model_zoo.NMR_gcn import NMR_GCN
from create_graph_data import create_graph
from node_embeddings import create_node_embeddings
from create_adjaency_matrix_from_labeled_pdb import build_adjacency_matrix
from dgl import save_graphs, load_graphs
from tqdm import tqdm

In [2]:
# from dgl.nn import GATv2Conv

# g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
# g = dgl.add_self_loop(g)
# feat = torch.ones(6, 10)
# gatv2conv = GATv2Conv(10, 2, num_heads=3)
# res = gatv2conv(g, feat)

In [3]:
from model_zoo.NMR_gat import GATv2

In [4]:
num_test = 462
# 399

atom_name_dim=256

bound_orig_dim=32 # not use

atom_type_dim=32 # not use

ab_dim=64

dl_dim=64

pf_dim=64

mono_accurate_dim=128 # not use

mono_simple_dim=64

me_dim=8

ser_dim=8

s_dim=8

ac_dim=16

gc_dim=8

use_atom = 1

use_mono=1

use_ab=1

use_dl=1

use_pf=1

use_inter=1


in_size = atom_name_dim * use_atom + ab_dim * use_ab + dl_dim * use_dl + \
          pf_dim * use_pf + mono_simple_dim * use_mono + \
        (me_dim + ser_dim + s_dim + ac_dim + gc_dim) * use_inter


hidden_size_1 = int(in_size / 2)
hidden_size_2 = int(hidden_size_1 / 2)
hidden_size_3 = 256
hidden_size_4 = 128

num_epoch = 3000
lr = 1e-3

# num_epoch = 5000
# lr = 1e-4

threshold_carbon=1.65
threshold_hydrogen=1.18
threshold_general=1.5
threshold_interaction=5.0


seed=97211


data_dir = 'dataset/Godess_final_data/Godess_data_rev/'


In [5]:
additional_drop = [221, 325, 2304, 1184, 1408, 1984, 1976, 1506, 525, 1531, 746, 2000, 1053, 1328, 2114, 94, 929, 
                  2134, 2287, 1575, 501, 1034, 839, 646, 1602, 2403, 1857]

In [6]:

# C = create_node_embeddings(data_dir = data_dir)
# C.write_all_embeddings(atom_name_dim = atom_name_dim, bound_orig_dim = bound_orig_dim, 
#                            atom_type_dim = atom_type_dim, ab_dim = ab_dim, dl_dim = dl_dim,
#                            pf_dim = pf_dim, mono_accurate_dim = mono_accurate_dim, 
#                            mono_simple_dim = mono_simple_dim, me_dim=me_dim, 
#                           ser_dim=ser_dim, s_dim=s_dim, ac_dim=ac_dim, gc_dim=gc_dim)

In [7]:
# B = build_adjacency_matrix(labeled_pdb_dir = data_dir,
#                            threshold_carbon=threshold_carbon,
#                            threshold_hydrogen=threshold_hydrogen, 
#                            threshold_general=threshold_general, 
#                            threshold_interaction=threshold_interaction)
# B.calculate_all_matrix()

In [8]:
# Create = create_graph(data_dir=data_dir, 
#                       num_test=num_test, 
#                       additional_drop=additional_drop, 
#                       seed=seed, 
#                       use_atom=use_atom, 
#                       use_mono=use_mono, 
#                       use_ab=use_ab, 
#                       use_dl=use_dl, 
#                       use_pf=use_pf, 
#                       use_inter=use_inter)
# g, test_index = Create.create_all_graph()

In [9]:
# save_graphs('dataset/Godess_final_data/graph/save_graph/data1.bin', g, {'feat': g.ndata['feat']})

# pd.DataFrame(g.ndata['train_carbon_mask'].cpu().numpy()).\
#     to_csv('dataset/Godess_final_data/graph/save_graph/train_carbon_mask.csv', index = False)

# pd.DataFrame(g.ndata['test_carbon_mask'].cpu().numpy()).\
#     to_csv('dataset/Godess_final_data/graph/save_graph/test_carbon_mask.csv', index = False)

# pd.DataFrame(g.ndata['shift_value'].cpu().numpy()).\
#     to_csv('dataset/Godess_final_data/graph/save_graph/shift_value.csv', index = False)

# pd.DataFrame(test_index).to_csv('dataset/Godess_final_data/graph/save_graph/test_index_experiment.csv', index=False)

In [10]:
# model =  GATv2(in_feats=in_size).to(device)
# sum(param.numel() for param in model.parameters())


In [11]:
g, features = load_graphs('dataset/Godess_final_data/graph/save_graph/data1.bin')
g = g[0]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)

# train_carbon_mask.to(device)
# test_carbon_mask.to(device)
# labels.to(device)
# features.to(device)

features = g.ndata["feat"]
labels = g.ndata["shift_value"]
masks = g.ndata['train_mask'], g.ndata['test_mask']


masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
# masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']
# masks = train_carbon_mask, test_carbon_mask

print(features.dtype)
print(labels.dtype)

model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)

# model =  GATv2(in_feats=in_size).to(device)


# NMR_prediction = NMR_prediction(results_dir='dataset/Godess_final_data/results/training_carbon.csv',
#                                 results_dir_test = 'dataset/Godess_final_data/results/testing_carbon.csv',
#                                 model_dir='dataset/Godess_final_data/results/Model_Godess_carbon.pt',
#                                 num_epoch = num_epoch, 
#                                 lr = lr)


NMR_prediction = NMR_prediction(results_dir='dataset/Godess_final_data/results/training_hydrogen.csv',
                                results_dir_test = 'dataset/Godess_final_data/results/testing_hydrogen.csv',
                                model_dir='dataset/Godess_final_data/results/Model_Godess_hydrogen.pt',
                                num_epoch = num_epoch, 
                                lr = lr)




print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  0%|                                          | 2/3000 [00:01<27:15,  1.83it/s]

19929
80921
Epoch 00000 | Loss 15.9860 | train_RMSE 3.1730 | test_RMSE 3.1712 
19929
80921
Epoch 00001 | Loss 10.0834 | train_RMSE 1.8140 | test_RMSE 1.8165 


  0%|                                          | 3/3000 [00:01<17:14,  2.90it/s]

19929
80921
Epoch 00002 | Loss 3.3887 | train_RMSE 1.0511 | test_RMSE 1.0364 
19929
80921
Epoch 00003 | Loss 1.4641 | train_RMSE 2.1101 | test_RMSE 2.0920 
19929


...

100%|██████████████████████████████████████▉| 2997/3000 [04:37<00:00, 10.78it/s]

19929
80921
Epoch 02996 | Loss 0.0276 | train_RMSE 0.1385 | test_RMSE 0.1354 
19929
80921
Epoch 02997 | Loss 0.0277 | train_RMSE 0.1368 | test_RMSE 0.1340 
19929


100%|███████████████████████████████████████| 3000/3000 [04:37<00:00, 10.80it/s]

80921
Epoch 02998 | Loss 0.0272 | train_RMSE 0.1377 | test_RMSE 0.1346 
19929
80921
Epoch 02999 | Loss 0.0270 | train_RMSE 0.1390 | test_RMSE 0.1359 
best loss: 0.02680950239300728
Testing...


80921
19929
train RMSE 0.1372
test RMSE 0.1343


In [ ]:
sum(p.numel() for p in saved_model.parameters())

In [ ]:
g.ndata['test_carbon_mask'].cpu().detach().numpy().sum()

In [ ]:
# df_visulize = pd.read_csv('dataset/Godess_final_data/results/training_carbon.csv')
# df_visulize = pd.read_csv('dataset/Godess_final_data/results/testing_carbon.csv')


# df_visulize = pd.read_csv('dataset/Godess_final_data/results/training_hydrogen.csv')
df_visulize = pd.read_csv('dataset/Godess_final_data/results/testing_hydrogen.csv')


data_dir = 'dataset/Godess_final_data/Godess_data_rev/'
files_labels_list = os.listdir(data_dir)

In [ ]:
test_index = list(pd.read_csv('dataset/Godess_final_data/graph/save_graph/test_index_experiment.csv')['0'])

In [ ]:
(np.array(test_index) == -1.0).sum()

In [ ]:
atom_test_list = []
residue_test_list = []
atom_name_test_list = []
residue_name_test_list = []
# hydrogen_list_1_6 = ['H1', 'H11', 'H2', 'H3', 'H31', 'H4', 'H5', 'H51', 'H6', 'H61', 'H7', 'H8', 'H9', 'H91']
# hydrogen_list_1_6 = ['H1', 'H11', 'H2', 'H3', 'H31', 'H4', 'H5', 'H51', 'H6', 'H61',
#                      'Neup_H2', 'Neup_H3', 'Neup_H31', 'Neup_H4', 'Neup_H5', 'Neup_H6', 
#                      'Neup_H7', 'Neup_H8', 'Neup_H9', 'Neup_H91']


hydrogen_list_1_6 = ['H1', 'H11', 'H2', 'H3', 'H4', 'H41', 'H5', 'H51', 'H6', 'H61',
                     'H7', 'Kdo_H3', 'Kdo_H31', 'Kdo_H4', 'Kdo_H5', 'Kdo_H6', 'Kdo_H7',
                     'Kdo_H8', 'Kdo_H81', 'Neup_H2', 'Neup_H3', 'Neup_H31', 'Neup_H4',
                     'Neup_H5', 'Neup_H6', 'Neup_H7', 'Neup_H8', 'Neup_H9', 'Neup_H91']

# carbon_list = ['C1', 'C11', 'C2', 'C21', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
# carbon_list = ['C1', 'C11', 'C2', 'C21', 'C3', 'C4', 'C5', 'C6',
#                'Neup_C1','Neup_C11', 'Neup_C2', 'Neup_C21', 'Neup_C3', 'Neup_C4',
#                'Neup_C5', 'Neup_C6', 'Neup_C7', 'Neup_C8', 'Neup_C9']

carbon_list = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'Kdo_C1',
               'Kdo_C2', 'Kdo_C3', 'Kdo_C4', 'Kdo_C5', 'Kdo_C6', 'Kdo_C7',
               'Kdo_C8', 'Neup_C1', 'Neup_C2', 'Neup_C3', 'Neup_C4', 'Neup_C5',
               'Neup_C6', 'Neup_C7', 'Neup_C8', 'Neup_C9']

for i in range(len(files_labels_list)):
    if test_index[i] == 1:
#     if test_index[i] == 0:
        temp_df = pd.read_csv(os.path.join(data_dir, files_labels_list[i]))
    
    
        temp_atom_list = list(temp_df.loc[(temp_df['main_ring_shift'] !=-1)]['New_Atom_name'].values)
        
        temp_atom_list_hydrogen = [i for i in temp_atom_list if i in hydrogen_list_1_6]
        atom_test_list.extend(temp_atom_list_hydrogen)
        
        atom_name_test_list.extend(np.repeat(files_labels_list[i], len(temp_atom_list_hydrogen)))
        
        
        temp_residue_list = temp_df.loc[(temp_df['main_ring_shift'] !=-1)]['Residual_accurate_name'].values
        
        temp_residue_list_hydrogen = []
        for m in range(len(temp_residue_list)):
            if temp_atom_list[m] in hydrogen_list_1_6:
                temp_residue_list_hydrogen.append(temp_residue_list[m])
        
        residue_test_list.extend(temp_residue_list_hydrogen)
        
        
        
        
#         temp_atom_list = list(temp_df.loc[(temp_df['main_ring_shift'] !=-1) & ((temp_df['main_ring_shift'] > 10))]['New_Atom_name'].values)
        
#         temp_atom_list_carbon = [i for i in temp_atom_list if i in carbon_list]
#         atom_test_list.extend(temp_atom_list_carbon)
        
        
#         atom_name_test_list.extend(np.repeat(files_labels_list[i], len(temp_atom_list_carbon)))
        
        
#         temp_residue_list = list(temp_df.loc[(temp_df['main_ring_shift'] !=-1) & ((temp_df['main_ring_shift'] > 10))]['Residual_accurate_name'].values)
        
#         temp_residue_list_carbon = []
#         for m in range(len(temp_residue_list)):
#             if temp_atom_list[m] in carbon_list:
#                 temp_residue_list_carbon.append(temp_residue_list[m])
        
#         residue_test_list.extend(temp_residue_list_carbon)

In [ ]:
len(atom_test_list), len(df_visulize), len(residue_test_list)

In [ ]:
df_visulize['atom_type'] = atom_test_list

df_visulize['residue_name'] = residue_test_list

df_visulize['glycan_name'] = atom_name_test_list

df_visulize['difference'] = np.abs(df_visulize['1'] - df_visulize['0'])

df_visulize = df_visulize.sort_values(by = ['difference'], ascending=False)

df_visulize.index = range(len(df_visulize))

In [ ]:
# df_visulize

In [ ]:
# df_visulize.to_csv('dataset/Godess_final_data/visulize/carbon/train_carbon.csv', index = False)

# df_visulize.to_csv('dataset/Godess_final_data/visulize/carbon/testing_carbon.csv', index = False) 

# df_visulize.to_csv('dataset/Godess_final_data/visulize/hydrogen/train_hydrogen.csv', index = False)

# df_visulize.to_csv('dataset/Godess_final_data/visulize/hydrogen/testing_hydrogen.csv', index = False)

In [ ]:
# df_visulize.to_csv('dataset/Godess_final_data/visulize/carbon/train_carbon_extra_feature.csv', index = False)

# df_visulize.to_csv('dataset/Godess_final_data/visulize/carbon/testing_carbon_extra_feature.csv', index = False) 

# df_visulize.to_csv('dataset/Godess_final_data/visulize/hydrogen/train_hydrogen_extra_feature.csv', index = False)

# df_visulize.to_csv('dataset/Godess_final_data/visulize/hydrogen/testing_hydrogen_extra_feature.csv', index = False)

In [ ]:
# np.unique(df_visulize['atom_type'])

In [ ]:
# num_test = 462
# # 399

# atom_name_dim=256

# bound_orig_dim=32

# atom_type_dim=32

# ab_dim=64

# dl_dim=64

# pf_dim=64

# mono_accurate_dim=128

# mono_simple_dim=256

# me_dim=32

# ser_dim=32

# s_dim=64

# ac_dim=64

# gc_dim=32


# in_size = atom_name_dim + bound_orig_dim + atom_type_dim + ab_dim + dl_dim + \
#             pf_dim + mono_accurate_dim + mono_simple_dim + me_dim + ser_dim + \
#             s_dim + ac_dim + gc_dim

# hidden_size_1 = int(in_size / 2)
# hidden_size_2 = int(hidden_size_1 / 2)
# hidden_size_3 = 256
# hidden_size_4 = 128

# num_epoch = 3000
# lr = 1e-3

# # num_epoch = 5000
# # lr = 1e-4

# threshold_carbon=1.65
# threshold_hydrogen=1.18
# threshold_general=1.5
# threshold_interaction=5.0


# seed=97211


# data_dir = 'dataset/Godess_final_data/Godess_data_rev/'
